# Fix Classes
version: 1

author: nuno costa

In [1]:
from annotate_v4 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle

#Define root dir dependent on OS
rdir='D:/external_datasets/' 
if str(platform.platform()).find('linux')>-1: rdir='/mnt/d/external_datasets/' 
print('OS: {}'.format(platform.platform()))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.20226-SP0
root dir: D:/external_datasets/


In [115]:
mlabjson =  json.load(open(rdir+'mlab.json'))

In [114]:
for k in mlabjson:
    print(k, len(mlabjson[k]))

info 5
licenses 9
categories 1310
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 1954769
datasets 2


### import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [132]:
# datasets name and id
dset_l=[]
dset_l_id=[]
for d in mlabjson['datasets']:
    dset_l.append(d['name'])
    dset_l_id.append(d['id'])
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [133]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in mlabjson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
#print(cat_l_id)

In [134]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in mlabjson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [122]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(mlabjson['annotations']):
    try:
        ann_catid.append(an['category_id'])
        ann_imgid.append(an['image_id'])
        ann_bbox.append(an['bbox'])
        ann_dset.append(an['dataset'])
    except:
        pass

100%|█████████████████████████████████████████████████████████████████| 1954769/1954769 [00:08<00:00, 242045.29it/s]


### 1. Find duplicates cat_ids
Duplicates example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [37]:
#duplicates #TODO: SORT alphabetically
duplicates_l=[]
duplicates_l_catid=[]
duplicates_l_catdset=[]
duplicate_method="similar_names"
if duplicate_method=="equal_names": #FIX REPETITIONS
    duplicates_l=list(set([x for x in cat_l if cat_l.count(x) > 1])) # duplicates l 
    for duplicate in tqdm(duplicates_l):
        idx_mask = [name == duplicate for name in cat_l] #mask of index of duplicate
        catids = np.array(cat_l_id)[idx_mask] #duplicate catids
        catdsets = np.array(cat_l_dset)[idx_mask] #duplicate catdset
        duplicates_l_catid.append(catids.tolist())
        duplicates_l_catdset.append(catdsets.tolist())
if duplicate_method=="similar_names": #FIX SIMILAR NAMES
    #WARNING: fix equal_names first - if not, it will not be fixed in this case
    import difflib
    for cat in cat_l: 
        match_l=[]
        match_l_temp=[c for c in cat_l if (c).find(cat)>-1] #only substring inside string - catagory with big name is not found
        if not len(match_l_temp)>1: continue #more than one
        #refine search #ADD or REMOVE refine options
        match_l.append(cat) #make category to be the first item
        match_l.extend([c for c in match_l_temp if ((c+' ').find(cat)>-1 or (c+' ').find(cat)>-1 or (' '+c).find(cat)>-1 or (c+'_').find(cat)>-1 or ('_'+c).find(cat)>-1 )]) #mantain optins: c+'_';'_'+c 
        match_l.extend(difflib.get_close_matches(cat, match_l_temp, n=5, cutoff=0.9)) #get similar words
        #remove equal names
        match_l=list(dict.fromkeys(match_l))#match_l=list(set(match_l))
        if not len(match_l)>1: continue #more than one
        #add to duplicates
        duplicates_l.append(match_l)
    for duplicates in tqdm(duplicates_l):
        catids=[]
        catdsets=[]
        for duplicate in duplicates:
            idx=cat_l.index(duplicate) #first duplicate ()
            catids.append(cat_l_id[idx]) #duplicate catids
            catdsets.append(cat_l_dset[idx]) #duplicate cat dsets
        duplicates_l_catid.append(catids)
        duplicates_l_catdset.append(catdsets)


print(duplicates_l)
print(duplicates_l_catid)
print(duplicates_l_catdset)
print(len(duplicates_l))
print(len(duplicates_l_catid))
print(len(duplicates_l_catdset))

100%|██████████████████████████████████████████████████████████████████████████| 189/189 [00:00<00:00, 12604.92it/s]

[['car', 'carrot', 'birthday_card', 'business_card', 'cabin_car', 'car_(automobile)', 'railcar_(part_of_a_train)', 'elevator_car', 'car_battery', 'identity_card', 'card', 'cardigan', 'cargo_ship', 'carnation', 'horse_carriage', 'cart', 'carton', 'credit_card', 'escargot', 'freight_car', 'golfcart', 'handcart', 'headscarf', 'keycard', 'passenger_car_(part_of_a_train)', 'playing_card', 'postcard', 'race_car', 'runner_(carpet)', 'scarecrow', 'scarf', 'shopping_cart', 'carbonated_water'], ['bus', 'bus_(vehicle)', 'business_card', 'school_bus'], ['train', 'bullet_train', 'railcar_(part_of_a_train)', 'passenger_car_(part_of_a_train)', 'strainer', 'train_(railroad_vehicle)'], ['truck', 'garbage_truck', 'pickup_truck', 'tow_truck', 'trailer_truck'], ['boat', 'fishing_boat', 'gondola_(boat)', 'gravy_boat', 'houseboat', 'motorboat', 'river_boat'], ['bench', 'pew_(church_bench)'], ['bird', 'birdfeeder', 'birdbath', 'birdcage', 'birdhouse', 'hummingbird', 'seabird', 'turkey_(bird)'], ['dog', 'hot 

In [38]:
# get annotations duplicates
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
duplicates_l_imgid=[]
duplicates_l_bbox=[]
duplicates_l_dset=[]
for catids in tqdm(duplicates_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    duplicates_l_imgid.append(l_imgid)
    duplicates_l_bbox.append(l_bbox)
    duplicates_l_dset.append(l_dset)

100%|█████████████████████████████████████████████████████████████████████████████| 189/189 [00:02<00:00, 89.19it/s]


### 2. Save classes to fix 

In [81]:
#INCEPTION Functions ===>dream within a dream =>>> e fez-se chockapic
def display_imgs(i, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0):
    for ii, did in enumerate(duplicates_l_catid[i]): #WARNING: only in duplicates each id should correspond to a different dataset
        print(dset_l[duplicates_l_dset[i][ii][imgidx]-1]) #WARNING: it works because mlabjson[datasets] is ordered
        imgid=duplicates_l_imgid[i][ii][imgidx]
        img_l_idx= img_l_id.index(imgid) #np.where(img_l_id_np==imgid)[0].tolist()
        imgpath=rdir+img_l[img_l_idx]
        print(imgpath)
        display(Image(imgpath))
        
def save_classtofix(i, classtofix_l, classtofix_l_catid, duplicates_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0):
    x = input('\n>> Choose: Rename Class and fix; 1=Fix only; 2=Do not fix; 3=show other pair; 4=Stop loop ')
    if x=='1': 
        classtofix_l.append(duplicates_l[i])
        classtofix_l_catid.append(duplicates_l_catid[i])
    elif x=='2': pass
    elif x=='3': 
        print('\n>> #WARNING showing another pair')
        imgidx+=1
        display_imgs(i, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=imgidx)
        classtofix_l, classtofix_l_catid=save_classtofix(i, classtofix_l, classtofix_l_catid, duplicates_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=imgidx)
    elif x=='4': raise RuntimeError('Stop')
    elif isinstance(x, str) and len(x)>2: #Rename only if string with more than 2 characters
        classtofix_l.append(x)
        classtofix_l_catid.append(duplicates_l_catid[i])
    else:
        print('\n>> #ERROR Try again! If Renaming use more than 2 characters! ')
        classtofix_l, classtofix_l_catid=save_classtofix(i, classtofix_l, classtofix_l_catid, duplicates_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=imgidx)

    return classtofix_l, classtofix_l_catid


def save_imgs(path, i, dset_l, duplicates_l, duplicates_l_catid, duplicates_l_bbox, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0, showimage=False):
    for ii, did in enumerate(duplicates_l_catid[i]): #WARNING: only in duplicates each id should correspond to a different dataset
        #category
        category=duplicates_l[i]
        if isinstance(category, list): 
            category=category[ii] #if similar names [cow, cowgirl]
            print(">>> finding similar category: ", category)
        #ANNOTATION exist?
        if not duplicates_l_imgid[i][ii]:
            #print("\n>> {} is missing from annotations dataset {}".format(category,dset_l[ii] ))
            print(">>>> #WARNING {} is missing from json annotations".format(category))
            print('annotations datasets: ', duplicates_l_dset[i][ii])
            print('annotations imgids: ', duplicates_l_imgid[i][ii])
            print('annotations bbox: ', duplicates_l_bbox[i][ii])
            continue
        dataset=dset_l[duplicates_l_dset[i][ii][imgidx]-1]  #  #WARNING: it works because mlabjson[datasets] is ordered
        print(dataset) 
        #image
        imgid=duplicates_l_imgid[i][ii][imgidx]
        bbox=duplicates_l_bbox[i][ii][imgidx]
        img_l_idx= img_l_id.index(imgid) #np.where(img_l_id_np==imgid)[0].tolist()
        imgpath=rdir+img_l[img_l_idx]
        #Copy and paste with bbox
        src=imgpath
        extension = os.path.splitext(src)[1]
        dst=path+dataset+'_'+category+'_'+str(imgid)+extension
        dpi = 80
        #IMAGE exist?
        try:
            #imgread
            img=imread(src)
            height, width, nbands = img.shape
            figsize = width / float(dpi), height / float(dpi) # What size does the figure need to be in inches to fit the image?
            #bbox
            x,y,w,h=bbox
            rect=Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
            #save fig
            fig = plt.figure(figsize=figsize) # Create a figure of the right size with one axes that takes up the full figure
            ax = fig.add_axes([0, 0, 1, 1])
            ax.axis('off') # Hide spines, ticks, etc.
            #fig,ax = plt.subplots(1)
            ax.imshow(img)
            ax.add_patch(rect)
            # Ensure we're displaying with square pixels and the right extent.
            # This is optional if you haven't called `plot` or anything else that might
            # change the limits/aspect.  We don't need this step in this case.
            ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)
            fig.savefig(dst, dpi=dpi, transparent=True)
            if showimage: plt.show()
            plt.ioff()
            plt.close()
        except FileNotFoundError as e:
            print(e)
            found=False
            for imgidx_n, imgid in enumerate(tqdm(duplicates_l_imgid[i][ii], desc='>>> Finding {} {}...'.format(dataset,category))):
                imgid=duplicates_l_imgid[i][ii][imgidx_n]
                bbox=duplicates_l_bbox[i][ii][imgidx_n]
                img_l_idx= img_l_id.index(imgid) #np.where(img_l_id_np==imgid)[0].tolist()
                imgpath=rdir+img_l[img_l_idx]
                src=imgpath
                try:
                    #imgread
                    img=imread(src)
                    found=True
                    break
                except:
                    continue
            if found:
                extension = os.path.splitext(src)[1]
                dst=path+dataset+'_'+category+'_'+str(imgid)+extension
                height, width, nbands = img.shape
                figsize = width / float(dpi), height / float(dpi) # What size does the figure need to be in inches to fit the image?
                #bbox
                x,y,w,h=bbox
                rect=Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
                #save fig
                fig = plt.figure(figsize=figsize) # Create a figure of the right size with one axes that takes up the full figure
                ax = fig.add_axes([0, 0, 1, 1])
                ax.axis('off') # Hide spines, ticks, etc.
                #fig,ax = plt.subplots(1)
                ax.imshow(img)
                ax.add_patch(rect)
                # Ensure we're displaying with square pixels and the right extent.
                # This is optional if you haven't called `plot` or anything else that might
                # change the limits/aspect.  We don't need this step in this case.
                ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)
                fig.savefig(dst, dpi=dpi, transparent=True)
                if showimage: plt.show()
                plt.ioff()
                plt.close()
            else:
                print('>>>> #WARNING: No file found for: {} {}...'.format(dataset,category))
            
def view_duplicate(i,ii,dataset,category,step=10,dpi=80):
    counter=0
    for imgidx, imgid in enumerate(tqdm(duplicates_l_imgid[i][ii], desc='>> Finding {} {}...'.format(dataset,category))):
        if imgidx==counter*10:
            imgid=duplicates_l_imgid[i][ii][imgidx]
            bbox=duplicates_l_bbox[i][ii][imgidx]
            img_l_idx= img_l_id.index(imgid) #np.where(img_l_id_np==imgid)[0].tolist()
            imgpath=rdir+img_l[img_l_idx]
            src=imgpath
            try:
                #imgread
                img=imread(src)
            except:
                continue
            height, width, nbands = img.shape
            figsize = width / float(dpi), height / float(dpi) # What size does the figure need to be in inches to fit the image?
            #bbox
            x,y,w,h=bbox
            rect=Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
            #fig
            fig = plt.figure(figsize=figsize) # Create a figure of the right size with one axes that takes up the full figure
            ax = fig.add_axes([0, 0, 1, 1])
            ax.axis('off') # Hide spines, ticks, etc.
            #fig,ax = plt.subplots(1)
            ax.imshow(img)
            ax.add_patch(rect)
            # Ensure we're displaying with square pixels and the right extent.
            # This is optional if you haven't called `plot` or anything else that might
            # change the limits/aspect.  We don't need this step in this case.
            ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)
            plt.show()
            counter+=1        

def convert_unicode(uni, method='liststr'):
    result = None
    if method == 'liststr':
        result = []
        try:
            result = [value.replace("'", "") for value in uni.strip('[]').split(', ')] #WARNING: ", " - space to split is necessary
        except:
            pass
    return result
    if method == 'listnum':
        result = []
        try:
            result = [float(value) for value in uni.strip('[]').split(',')] #uni.strip('[]').split(',') #[value for value in uni.strip('[]').split(',')]
        except:
            pass
    return result
    
    if method == 'listoflistnum':
        result = []
        try:
            result = [[float(value.replace('[', '')) for value in epoch.split(',')] for epoch in uni.strip('[]').split('],')]
        except:
            pass
    return result

def parse_path(path):
    """
    Python only works with '/', not '\\'or '\'

    WARNING:
        in windows use r'path' because of escape literals , e.g: "."
        os.path.realpath(path).replace('\\', '/') #BUG os.path.realpath removes the last '\\' and if your sending a folder it is a problem

    """
    parsed_path = path.replace('\\', '/')
    parsed_path = parsed_path.replace("\ ", '/')
    return parsed_path

def assure_path_exists(path):
    """
    Create directory folders from path

    Note: windows path strings should be r'path'
    """

    #Only for dirs - for complete you have to change dir for path
    dir_p = os.path.normpath(path) #dirname is obrigatory - make sure it is a dir
    if not os.path.exists(dir_p):
        os.makedirs(dir_p)

In [41]:
#class to fix: options yes, no and rename
classtofix_l=[]
classtofix_l_catid=[]
method="save_images"
if method=="all": #fix all duplicates
    classtofix_l=duplicates_l
    classtofix_l_catid=duplicates_l_catid
if method=="view_images": # view images and select class to fix (INCEPTION =>>>Pufff!!!!)
    img_l_id_np=np.array(img_l_id)      
    for i, duplicate in enumerate(tqdm(duplicates_l)): #run for each duplicate
        print(duplicate+'...') #class
        display_imgs(i, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0)
        classtofix_l, classtofix_l_catid=save_classtofix(i, classtofix_l, classtofix_l_catid, duplicates_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0)
if method=="save_images": # save images to folder for manual check
    datadir="duplicates"
    folder=duplicate_method
    showimage=False #show images
    imgidx=0 # index of image to save from each dataset
    path=os.path.join(rdir,datadir,folder)
    assure_path_exists(path)
    #TODO
    #save excel # TODO Send duplicates_l, duplicates_L_catid, duplicates_l_catdset: user should make column  classtofix_l and classtofix_L_catid - user should create  
    df=pd.DataFrame({'duplicates_l': duplicates_l,'duplicates_l_catid': duplicates_l_catid, 'duplicates_l_catdset': duplicates_l_catdset, 'classtofix_l': np.nan, 'classtofix_l_catid':np.nan})
    excelpath=path+'/'+"classtofix.xlsx"
    df.to_excel(excelpath, index=False)
    """
    df=pd.DataFrame({'originalcatlabel': duplicates_l,'merge': np.ones(len(duplicates_l)).tolist(), 'remove': np.zeros(len(duplicates_l)).tolist(), 'newcatlabel': np.nan})
    for d in dset_l:
        df[d+'_newcatlabel']=np.nan
    excelpath=path+'/'+"classtofix.xlsx"
    df.to_excel(excelpath, index=False)
    """
    #save image for each duplicate
    for i, duplicate in enumerate(tqdm(duplicates_l)): #run for each duplicate category
        minclass=duplicate
        if isinstance(minclass, list): minclass=minclass[0]
        print('\n>> '+minclass+'...') #class
        classpath=os.path.join(path, minclass) # folder for images of this class
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        save_imgs(classpath, i, dset_l, duplicates_l, duplicates_l_catid, duplicates_l_bbox, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=imgidx, showimage=showimage)
if method=="view_duplicate":
    i=-1 #duplicate_l index
    ii=1 #dset_l index
    step=10 #step images
    dpi=80
    dataset=dset_l[ii]
    category=duplicates_l[i]
    view_duplicate(i,ii,dataset,category,step=step,dpi=dpi)

>>> Finding TAO business_card...: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 312.50it/s]



>> car...
>>> finding similar category:  car
COCO
>>> finding similar category:  carrot
COCO
>>> finding similar category:  birthday_card
>>>> #WARNING birthday_card is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  business_card
TAO
[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/AVA/4Y5qi1gD2Sw_scene_1_28435-31083/frame1081.jpg'
>>>> #WARNING: No file found for: TAO business_card...
>>> finding similar category:  cabin_car
>>>> #WARNING cabin_car is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  car_(automobile)
TAO



>>> Finding TAO railcar_(part_of_a_train)...: 100%|████████████████████████████████| 20/20 [00:00<00:00, 333.34it/s]


>>> finding similar category:  railcar_(part_of_a_train)
TAO
[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/YFCC100M/v_513e50c31f11f0bdbeb599a811aa1413/frame1033.jpeg'
>>>> #WARNING: No file found for: TAO railcar_(part_of_a_train)...
>>> finding similar category:  elevator_car
>>>> #WARNING elevator_car is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  car_battery
TAO
>>> finding similar category:  identity_card
>>>> #WARNING identity_card is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  card
>>>> #WARNING card is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  cardigan
>>>> #WARNING cardigan is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>

>>> Finding TAO business_card...: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 333.64it/s]

...

>>> finding similar category:  tow_truck
>>>> #WARNING tow_truck is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  trailer_truck
TAO

>> vat...
>>> finding similar category:  vat
>>>> #WARNING vat is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  elevator_car
>>>> #WARNING elevator_car is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> turtle...
>>> finding similar category:  turtle
TAO
>>> finding similar category:  turtleneck_(clothing)
>>>> #WARNING turtleneck_(clothing) is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> urn...


 98%|███████████████████████████████████████████████████████████████████████████▊ | 186/189 [00:32<00:00,  5.37it/s]

>>> finding similar category:  urn
>>>> #WARNING urn is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  chest_of_drawers_(furniture)
>>>> #WARNING chest_of_drawers_(furniture) is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  turnip
>>>> #WARNING turnip is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> waffle...
>>> finding similar category:  waffle
>>>> #WARNING waffle is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  waffle_iron
>>>> #WARNING waffle_iron is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wagon...
>>> finding similar category:  wagon
>>>> #WARNING wagon is missing from json annotations
annotations dat

100%|█████████████████████████████████████████████████████████████████████████████| 189/189 [00:32<00:00,  5.81it/s]

>>> finding similar category:  wagon_wheel
>>>> #WARNING wagon_wheel is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  wheelchair
>>>> #WARNING wheelchair is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yak...
>>> finding similar category:  yak
>>>> #WARNING yak is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []
>>> finding similar category:  kayak
TAO


In [88]:
#WARNING: CHECK EXCEL FIRST
if method=="save_images": # reading classtofix from excel
    classtofix_l=[]
    classtofix_l_catid=[]
    df=pd.read_excel(excelpath)
    classtofix_df=df.loc[:,'classtofix_l']
    classtofix_df_catid=df.loc[:,'classtofix_l_catid']
    display(df)
    """
    #print(df)
    for index, row in df.iterrows():
        assert row['originalcatlabel']==duplicates_l[index]
        if row['remove']==1: continue #TODO - needs to be implemented for removing - for now it does not merge
        if row['merge']==1:
            label=row['originalcatlabel']
            if not np.isnan(row['newcatlabel']): label=row['newcatlabel']
            classtofix_l.append(label)
        elif row['merge']==0:
            if all([np.isnan(row[dset+'_newcatlabel']) for dset in dset_l]): continue #don't do nothing
            label=row['originalcatlabel']
            label_l=[]
            for dset in dset_l:
                if not np.isnan(row[dset+'_newcatlabel']): label=row[dset+'_newcatlabel']
                label_l.append(label)
            classtofix_l.append(label)
        classtofix_l_catid.append(duplicates_l_catid[index])
    """
                            

,duplicates_l,duplicates_l_catid,duplicates_l_catdset,classtofix_l,classtofix_l_catid
0,"['car', 'carrot', 'birthday_card', 'business_c...","[3, 52, 185, 255, 262, 291, 292, 293, 294, 295...","['COCO', 'COCO', 'TAO', 'TAO', 'TAO', 'TAO', '...","['car', 'car_(automobile)']",NaN
1,"['bus', 'bus_(vehicle)', 'business_card', 'sch...","[6, 254, 255, 1016]","['COCO', 'TAO', 'TAO', 'TAO']",NaN,NaN
2,"['train', 'bullet_train', 'railcar_(part_of_a_...","[7, 245, 292, 861, 1121, 1215]","['COCO', 'TAO', 'TAO', 'TAO', 'TAO', 'TAO']",NaN,NaN
3,"['truck', 'garbage_truck', 'pickup_truck', 'to...","[8, 570, 893, 1207, 1214]","['COCO', 'TAO', 'TAO', 'TAO', 'TAO']",NaN,NaN
4,"['boat', 'fishing_boat', 'gondola_(boat)', 'gr...","[9, 536, 592, 601, 665, 793, 981]","['COCO', 'TAO', 'TAO', 'TAO', 'TAO', 'TAO', 'T...",NaN,NaN
...,...,...,...,...,...
184,"['waffle', 'waffle_iron']","[1251, 1252]","['TAO', 'TAO']",NaN,NaN
185,"['wagon', 'wagon_wheel']","[1253, 1254]","['TAO', 'TAO']",NaN,NaN
186,"['watch', 'pocket_watch']","[1263, 917]","['TAO', 'TAO']",NaN,NaN
187,"['wheel', 'Ferris_wheel', 'pinwheel', 'steerin...","[1281, 520, 902, 1108, 1254, 1282]","['TAO', 'TAO', 'TAO', 'TAO', 'TAO', 'TAO']",NaN,NaN


In [92]:
classtofix_l=classtofix_df.tolist()
classtofix_l_catid=classtofix_df_catid.tolist()
#convert strings lists
for icl, cl in enumerate(classtofix_l): 
    if isinstance(classtofix_l[icl], str): classtofix_l[icl]=convert_unicode(classtofix_l[icl], method='liststr')
    if isinstance(classtofix_l_catid[icl], str): classtofix_l_catid[icl]=convert_unicode(classtofix_l_catid[icl], method='liststr')
#get classtofix_l_catid if the column is null and classtofix_l has at least a value    
if classtofix_df_catid.isnull().sum() == len(classtofix_df_catid) and classtofix_df.isnull().sum() != len(classtofix_df) :
    for ic, classes in enumerate(classtofix_df):
        try:
            if classes.isnull(): continue
        except:
            pass
        classes=convert_unicode(classes, method='liststr')
        cids=[]
        for c in classes:
            cidx=duplicates_l[ic].index(c)
            cid=duplicates_l_catid[ic][cidx]
            cids.append(cid)
        classtofix_l_catid[ic]=cids
        


In [113]:
print('>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! \n')
fixempty=True
if fixempty:
    classtofix_l=[x for x in classtofix_l if str(x) != 'nan' and str(x) !='[]']
    classtofix_l_catid=[x for x in classtofix_l_catid if x != 'nan' and str(x) !='[]']
print(classtofix_l)
print(classtofix_l_catid)

>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! 

[['car', 'car_(automobile)']]
[[3, 291]]


### 3. Fix classes



In [18]:
newjson=copy.deepcopy(mlabjson) #do deepcopy to compare

In [135]:
classtofix_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtofix_l_catid]
#print(classtofix_l_catidx) # they should be less one, becacuse it is ordered
print(len(classtofix_l_catidx))

1


#### Change mlabjson['categories']: [{name: , id: }]  
=>  use min cat id and remove the other categories (!!!Without ordering again the category id!!!)

In [24]:
removeidx_l=[]
for i,id_l in enumerate(tqdm(classtofix_l_catid)): #for each classtofix
    minidx=0 # get first cat: minimum in equal names and the category used to search in similar_names
    minid=id_l[minidx] # #category id 
    catidx=classtofix_l_catidx[i][minidx]# get cat index of min catid
    newjson['categories'][catidx]['name']=classtofix_l[i] #change name of min id (if changed)
    assert newjson['categories'][catidx]['id']==id_l[minidx] #assert id - it should be the same
    otheridx_l=copy.copy(classtofix_l_catidx[i]) #the idx to remove
    otheridx_l.remove(catidx)
    removeidx_l.extend(otheridx_l) #remove index
removeidx_l=list(dict.fromkeys(removeidx_l)) # no duplicates

100%|██████████████████████████████████████████████████████████████████████████| 189/189 [00:00<00:00, 94630.95it/s]


In [26]:
#REMOVE - Newjson will be changed
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #items to remove
for removeitem in removeitem_l: newjson['categories'].remove(removeitem) 

In [27]:
#TEST
print(mlabjson['categories'][13])
print(newjson['categories'][13])
print(mlabjson['categories'][170])
print(newjson['categories'][170])

{'supercategory': 'outdoor', 'id': 14, 'name': 'bench', 'dataset': 1}
{'supercategory': 'animal', 'id': 15, 'name': ['bird', 'birdfeeder', 'birdbath', 'birdcage', 'birdhouse', 'hummingbird', 'seabird', 'turkey_(bird)'], 'dataset': 1}
{'frequency': 'f', 'id': 171, 'synset': 'bench.n.01', 'image_count': 93, 'instance_count': 160, 'synonyms': ['bench'], 'def': 'a long seat for more than one person', 'name': 'bench', 'dataset': 2}
{'frequency': 'c', 'id': 257, 'synset': 'butter.n.01', 'image_count': 11, 'instance_count': 26, 'synonyms': ['butter'], 'def': 'an edible emulsion of fat globules made by churning milk or cream; for cooking and table use', 'name': ['butter', 'butterfly', 'peanut_butter'], 'dataset': 2}


#### change mlabjson['annotations']: [{category_id: , }] 
=> remove_catid_l to annidx_l

In [28]:
ann_catid_np=np.array(ann_catid)
classtofix_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtofix_l_catid]
print(len(classtofix_l_ann_catidx))
print(classtofix_l_catidx[0][0])
print(len(classtofix_l_ann_catidx[0][0]))
print((classtofix_l_ann_catidx[0][0][0]))#to maintain same id
print((classtofix_l_ann_catidx[0][1][0]))#to change same id

189
2
45799
2584
10768


In [29]:
for i,id_l in enumerate(classtofix_l_catid): #for each classtofix
    minidx=0 # get first catid min
    minid=id_l[minidx]
    ann_catidx= classtofix_l_ann_catidx[i][minidx]# get annotation cat index of min catid
    ann_otheridx_l=copy.copy(classtofix_l_ann_catidx[i])
    ann_otheridx_l.remove(ann_catidx) #the idx to change
    for ann_otheridx in ann_otheridx_l[0]: newjson['annotations'][ann_otheridx]['category_id']=minid
        

In [30]:
#TEST
print(mlabjson['annotations'][8875])
print(newjson['annotations'][8875])
print(mlabjson['annotations'][1174300])
print(newjson['annotations'][1174300])

{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7, 105.73]], 'area': 5697.957150000001, 'iscrowd': 0, 'image_id': 26929, 'bbox': [8.43, 35.62, 127.08, 74.5], 'category_id': 14, 'id': 8876, 'dataset': 1}
{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7

### 4. Save fixed json

In [ ]:
# save
print('\n >> SAVING...')
jsonfile=rdir+'mlabfixed.json'
with open(jsonfile, 'w') as f:
    json.dump(newjson, f)
print("JSON SAVED : {} \n".format(jsonfile))

# BUG: Why image without category?

## Verificação das anotações no TAO
#NOTE: https://github.com/TAO-Dataset/tao/blob/master/docs/faqs.md
Why does the training set only contain 216 LVIS categories?

TAO contains a total of 482 LVIS categories. However, not all categories are present in the train, val, and test sets. Instead, we encourage researchers to train detectors on the LVIS v0.5 dataset, which contains a superset of the 482 categories, and trackers on existing single-object tracking datasets. TAO is primarily a benchmark dataset, but we provide a small set of training videos for tuning trackers.

In [ ]:
#mlab.json
cat_id_TAO=[]
for an in tqdm(mlabjson['annotations']):
    if an['dataset']==2 : cat_id_TAO.append(an['category_id'])

cat_id_TAO=list( dict.fromkeys(cat_id_TAO) )
cat_id_TAO.sort()

cat_l_idx= [cat_names_l_id.index(cat_id) for cat_id in cat_id_TAO]
cat_l=[cat_names_l[i] for i in cat_l_idx]
print(len(cat_id_TAO))
print(cat_l) #

In [ ]:
#val.json https://github.com/TAO-Dataset/annotations
taojson =  json.load(open(rdir+'TAO/TAO_DIR/annotations/val.json'))
cat_names_l_tao=[]
cat_names_l_id_tao=[]
for t in taojson['categories']:
    cat_names_l_tao.append(t['name'])
    cat_names_l_id_tao.append(t['id'])
print(len(cat_names_l_tao))

cat_id_TAO=[]
for an in tqdm(taojson['annotations']):
    cat_id_TAO.append(an['category_id'])

cat_id_TAO=list( dict.fromkeys(cat_id_TAO) )
cat_id_TAO.sort()

cat_l_idx= [cat_names_l_id_tao.index(cat_id) for cat_id in cat_id_TAO]
cat_l=[cat_names_l_tao[i] for i in cat_l_idx]
print(len(cat_id_TAO))
print(cat_l) #

In [ ]:
#train.json https://github.com/TAO-Dataset/annotations
taojson =  json.load(open(rdir+'TAO/TAO_DIR/annotations/train.json'))
cat_names_l_tao=[]
cat_names_l_id_tao=[]
for t in taojson['categories']:
    cat_names_l_tao.append(t['name'])
    cat_names_l_id_tao.append(t['id'])
print(len(cat_names_l_tao))

cat_id_TAO=[]
for an in tqdm(taojson['annotations']):
    cat_id_TAO.append(an['category_id'])

cat_id_TAO=list( dict.fromkeys(cat_id_TAO) )
cat_id_TAO.sort()

cat_l_idx= [cat_names_l_id_tao.index(cat_id) for cat_id in cat_id_TAO]
cat_l=[cat_names_l_tao[i] for i in cat_l_idx]
print(len(cat_id_TAO))
print(cat_l) #